In [6]:
%%capture

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane

# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

In this tutorial we will:

* learn step-by-step how quantum computations are implemented in PennyLane,
* understand parameter-dependent quantum computations ("variational circuits"), 

We need the following imports:

In [7]:
import pennylane as qml
from pennylane import numpy as np

## 1. Quantum nodes

In PennyLane, a *quantum node* is a computational unit that involves the construction, evaluation, pre- and postprocessing of quantum computations.

A quantum node consists of a *quantum function* that defines a circuit, as well as a *device* on which it is run. 

There is a growing [device ecosystem](https://pennylane.ai/plugins.html) which allows you to change only one line of code to dispatch your quantum computation to local simulators, remote simulators and remote hardware from different vendors.

Here we will use the built-in `default.qubit` device.

In [8]:
# demo_device = qml.device('default.qubit', wires=2)

from pennylane_quantuminspire2.qi_device import QI2Device

In [9]:
QI2Device.backends()

In [11]:
for backend in QI2Device.backends():
    print(f"{backend.name}")

Spin
Stubbed
QX emulator
6D2S simulator
HectoQubit/2
Starmon 7


In [12]:
backend = QI2Device.get_backend("Stubbed")

In [13]:
demo_device = QI2Device(backend=backend)

To combine the device with a quantum function to a quantum node we can use the `qml.qnode` decorator. The function can then be evaluated as if it was any other python function. Internally, it will construct a circuit and run it on the device.

In [14]:
# Define node

@qml.qnode(demo_device)
def my_quantum_circuit(circuit_params):
    qml.RX(circuit_params[0], wires=0)  # Apply an RX gate to qubit 0
    qml.RY(circuit_params[1], wires=1)  # Apply an RY gate to qubit 1
    qml.CNOT(wires=[0, 1])  # Apply a CNOT gate
    return qml.expval(qml.PauliZ(0))  # Measure the expectation value of PauliZ on qubit 0

In [15]:
# Execute the circuit

params = np.array([0.1, 0.2], requires_grad=True)
result = my_quantum_circuit(params)

In [16]:
result

0.0

In [17]:
# Perform optimization (optional)
# For example, use gradient descent to minimize the output

opt = qml.GradientDescentOptimizer(stepsize=0.1)
for i in range(1):
    params = opt.step(my_quantum_circuit, params)
result = my_quantum_circuit(params)
print(f"Optimized params: {params}")
print(f"Optimized result: {result}")

Optimized params: [0.1 0.2]
Optimized result: 0.0


## 2. Building quantum circuits

### The initial state

<br />
<img src="figures/1.png" width="500" height="100">
<br />

The initial state has 100% probability to be measured in the "0..0" configuration. Let's see how we can verify this with PennyLane.

In [ ]:
@qml.qnode(dev)
def circuit():
    return qml.probs(wires=[0, 1])

circuit()

The internal state vector that we use to mathematically keep track of probabilities is complex-valued. Since `default.qubit` is a simulator we can have a look at the state, for example by checking the device's `state` attribute.

In [ ]:
dev.state

### Unitary evolutions

<br />
<img src="figures/2.png" width="500">
<br />

Quantum circuits are represented by unitary matrices. We can evolve the initial state by an arbitrary unitrary matrix as follows:

In [ ]:
s = 1/np.sqrt(2)
U = np.array([[0., -s, 0.,  s],
              [ s, 0., -s, 0.],
              [ s, 0.,  s, 0.],
              [0., -s, 0., -s]])

@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.probs(wires=[0, 1])

circuit()

The internal quantum state changed.

In [ ]:
dev.state

### Measurements sample outcomes from the distribution

<br />
<img src="figures/3.png" width="500">
<br />

The most common measurement takes samples $-1, 1$ from the "Pauli-Z" observable. The samples indicate if the qubit was measured in state $| 0 \rangle$ or $| 1 \rangle$.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1))

circuit()

The quantum state should be still the same as above.

In [ ]:
dev.state

### Computing expectation values 

<br />
<img src="figures/4.png" width="500">
<br />

When we want outputs of computations to be deterministic, we often interpret the expected measurement outcome as the result. This value is estimated by taking lots of samples and averaging over them.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

Again, the quantum state should be the same as above.

In [ ]:
dev.state

### Quantum circuits are decomposed into gates

<br />
<img src="figures/5.png" width="500">
<br />

Quantum circuits rarely consist of one large unitary (which quickly becomes intractably large as the number of qubits grow). Instead, they are composed of *quantum gates*.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

### Some gates depend on "control" parameters

<br />
<img src="figures/6.png" width="500">
<br />

To train circuits, there is a special subset of gates which is of particular interest: the Pauli rotation gates. These "rotate" a special representation of the quantum state around a specific axis. The gates depend on a scalar parameter which is the angle of the rotation. 

In [ ]:
@qml.qnode(dev)
def circuit(w1, w2):
    qml.RX(w1, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(w2, wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit(1.2, 1.3)

The names `w1`, `w2` are already suggestive that these can be used like the trainable parameters of a classical machine learning model. But we could also call the control parameters `x1`, `x2` and encode data features into quantum states. 

## 3. A full quantum machine learning model and its gradient

Finally, we can use pre-coded routines or [templates](https://pennylane.readthedocs.io/en/stable/introduction/templates.html) to conveniently build full quantum machine learning model that include a data encoding part, and a trainable part.

<br />
<img src="figures/7.png" width="500">
<br />

Here, we will use the `AngleEmbedding` template to load the data, and the `BasicEntanglingLayers` as the trainable part of the circuit.

In [ ]:
@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))


x = np.array([0.1, 0.2], requires_grad=False)
w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])

quantum_model(x, w)

We can draw the circuit.

In [ ]:
print(quantum_model.draw())

The best thing is that by using PennyLane, we can easily compute its gradient!

In [ ]:
gradient_fn = qml.grad(quantum_model)

gradient_fn(x, w)

This allows us to slot the quantum circuit into the machine learning example from the previous notebook.

#  TASKS 

1. Copy and paste the code from the previous notebook to here and replace the classical model by 
   the `quantum_model` function. This will allow you to train the model!

2. Add a bias term to the quantum model.

A potential solution of 1 & 2 together:

In [ ]:
n_samples = 100
X0 = np.array([[np.random.normal(loc=-1, scale=1), 
                np.random.normal(loc=1, scale=1)] for i in range(n_samples//2)]) 
X1 = np.array([[np.random.normal(loc=1, scale=1), 
                np.random.normal(loc=-1, scale=1)] for i in range(n_samples//2)]) 

X = np.concatenate([X0, X1], axis=0)
Y = np.concatenate([-np.ones(50), np.ones(50)], axis=0)
data = list(zip(X, Y))

def model_with_bias(x, w, b):
    return quantum_model(x, w) + b

def loss(a, b):
    return (a - b)**2

def average_loss(w, b, data):
    c = 0
    for x, y in data:
        prediction = model_with_bias(x, w, b)
        c += loss(prediction, y)
    return c/len(data)

gradient_fn_w = qml.grad(average_loss, argnum=0)
gradient_fn_b = qml.grad(average_loss, argnum=1)

w_init = np.random.random(size=(3, 2))
w = np.array(w_init)
b = np.array(0.0) # start with zero bias

for i in range(15):
    w_new = w - 0.05*gradient_fn_w(w, b, data)
    b_new = b - 0.05*gradient_fn_b(w, b, data)
    print(average_loss(w_new, b_new, data))
    w = w_new
    b = b_new

3. Replace the hand-coded optimisation step by a native [PennyLane optimiser](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html).

In [ ]:
opt = qml.GradientDescentOptimizer(0.05)
for i in range(15):
    
    ([w, b], cst) = opt.step_and_cost(lambda w_, b_: average_loss(w_, b_, data), w, b)
    print(cst)

4. Rewrite the entire example in PyTorch. 

   Tipp: You must set the qnode to the correct interface via `@qml.qnode(dev, interface='tf')`.

In [ ]:
import torch

data = [[torch.tensor(x), torch.tensor(y)] for x, y in data]


@qml.qnode(dev, interface='torch')
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))

def average_loss(w, data):
    c = 0
    for x, y in data:
        prediction = quantum_model(x, w)
        c += loss(prediction, y)
    return c/len(data)

w_init = np.random.random(size=(3, 2))

w = torch.tensor(w_init, requires_grad=True)

opt = torch.optim.Adam([w], lr = 0.1)

# One way of optimising is to use closures
def closure():
    opt.zero_grad()
    loss = average_loss(w, data)
    loss.backward()
    return loss

for i in range(15):
    opt.step(closure)
    
    print(average_loss(w, data))
    